In [2]:
import pandas as pd
import numpy as np

In [4]:
lb = pd.read_csv('../data/London_Bridge.txt')

In [6]:
df = lb.iloc[:,0:3]
df.head()

,Date and time,water level (m ODN),flag
0,01/05/1911 15:40:00,3.7130,1
1,02/05/1911 11:25:00,-2.9415,0
2,02/05/1911 16:05:00,3.3828,1
3,03/05/1911 11:50:00,-2.6367,0
4,03/05/1911 16:55:00,2.9256,1


In [8]:
df.set_axis(['datetime','water_level','is_high_tide'],axis='columns',inplace=True)

/var/folders/wj/b4h4w1f97bs9bhrqtx86sr4w0000gn/T/ipykernel_71558/3332637567.py:1: FutureWarning: DataFrame.set_axis 'inplace' keyword is deprecated and will be removed in a future version. Use `obj = obj.set_axis(..., copy=False)` instead
  df.set_axis(['datetime','water_level','is_high_tide'],axis='columns',inplace=True)


In [13]:
df.head()

,datetime,water_level,is_high_tide
0,1911-01-05 15:40:00,3.7130,1
1,1911-02-05 11:25:00,-2.9415,0
2,1911-02-05 16:05:00,3.3828,1
3,1911-03-05 11:50:00,-2.6367,0
4,1911-03-05 16:55:00,2.9256,1


In [12]:
df.dtypes

datetime        datetime64[ns]
water_level             object
is_high_tide             int64
dtype: object

In [14]:
df['datetime'] = pd.to_datetime(df['datetime'])
df['water_level'] = df['water_level'].astype('float')

In [21]:
df['month'] = df.datetime.apply(lambda x: x.month)
df['year'] = df.datetime.apply(lambda x: x.year)

# Create extra month and year columns for easy access
#df['month'] = df['datetime'].dt.month
#df['year'] = df['datetime'].dt.year

In [22]:
df.head()

,datetime,water_level,is_high_tide,month,year
0,1911-01-05 15:40:00,3.7130,1,1,1911
1,1911-02-05 11:25:00,-2.9415,0,2,1911
2,1911-02-05 16:05:00,3.3828,1,2,1911
3,1911-03-05 11:50:00,-2.6367,0,3,1911
4,1911-03-05 16:55:00,2.9256,1,3,1911


In [31]:
# Filter df for high and low tide
tide_high = df.query('is_high_tide==1')['water_level']
tide_low = df.query('is_high_tide==0')['water_level']
#df.iloc[np.where(df['is_high_tide']==1)]['water_level']

In [36]:
# Create summary statistics
distributions = {
    'tide_high':{'mean':round(tide_high.mean(),2),'median':round(tide_high.median(),2),
                'interquartile_range':round((tide_high.quantile(0.75)-tide_high.quantile(0.25)),2)},
    
    'tide_low':{'mean':round(tide_low.mean(),2),'median':round(tide_low.median(),2),
                'interquartile_range':round((tide_low.quantile(0.75)-tide_low.quantile(0.25)),2)}

}
distributions

{'tide_high': {'mean': 3.32, 'median': 3.35, 'interquartile_range': 0.74},
 'tide_low': {'mean': -2.38, 'median': -2.41, 'interquartile_range': 0.54}}

In [44]:
all_high_days = df.query('is_high_tide==1').groupby('year').count()['water_level']
all_high_days

year
1911    244
1912    557
1913    669
1914    687
1915    666
       ... 
1991    706
1992    707
1993    699
1994    705
1995    705
Name: water_level, Length: 85, dtype: int64

In [50]:
high_75_days = df.loc[df.water_level>tide_high.quantile(0.75)].groupby('year').count()['water_level']
high_75_days

year
1911      8
1912     71
1913    125
1914    111
1915    146
       ... 
1991    178
1992    188
1993    222
1994    252
1995    229
Name: water_level, Length: 85, dtype: int64

In [57]:

high_ratio = (high_75_days/all_high_days).reset_index()


,year,water_level
0,1911,0.032787
1,1912,0.127469
2,1913,0.186846
3,1914,0.161572
4,1915,0.219219
...,...,...
80,1991,0.252125
81,1992,0.265912
82,1993,0.317597
83,1994,0.357447


In [58]:
all_low_days = df.loc[df.is_high_tide==0].groupby('year').count()['water_level']
all_low_days

year
1911    231
1912    555
1913    670
1914    692
1915    658
       ... 
1991    705
1992    707
1993    701
1994    705
1995    705
Name: water_level, Length: 85, dtype: int64

In [59]:
low_25_days = df.loc[df.water_level<tide_low.quantile(0.25)].groupby('year').count().water_level
low_25_days

year
1911     47
1912    107
1913     69
1914     98
1915     92
       ... 
1991    220
1992    188
1993    177
1994    178
1995    174
Name: water_level, Length: 85, dtype: int64

In [60]:
low_ratio = (low_25_days/all_low_days).reset_index()
low_ratio

,year,water_level
0,1911,0.203463
1,1912,0.192793
2,1913,0.102985
3,1914,0.141618
4,1915,0.139818
...,...,...
80,1991,0.312057
81,1992,0.265912
82,1993,0.252496
83,1994,0.252482
